# MÔ HÌNH: DỰ BÁO DOANH THU (REVENUE/DEMAND FORECAST)

## Mục tiêu bài toán
Dự báo doanh thu (hoặc số lượng đơn) theo thời gian, có thể theo tổng hệ thống hoặc theo từng quốc gia (Country) để:
- Kế hoạch doanh số (sales planning)
- Tối ưu hóa inventory
- Phân bổ resources
- Budget planning

## Input (Dữ liệu)
- **File nguồn:** `data/merged_supply_weather_clean.parquet`
- **Aggregation:** Group by `year_month` + `Order Country`
- **Các nhóm feature:** Lag features, Rolling stats, Time, Weather, Country

## Output (Yêu cầu dự đoán)
- **Target:** `revenue` (tổng Sales trong tháng)
- **Output:** Doanh thu dự báo, khoảng tin cậy

## Thông tin phiên bản
- **Ngày:** 2024
- **Phiên bản:** 1.0
- **Dataset:** merged_supply_weather_clean.parquet


In [ ]:
# Import thư viện & cấu hình chung
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import xgboost as xgb
import os
import warnings
warnings.filterwarnings('ignore')

# Cấu hình matplotlib
%matplotlib inline
plt.style.use('default')
sns.set_palette("husl")

# Random seed để đảm bảo reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Đã import thư viện và cấu hình xong")


## 1. Load dữ liệu

Load dữ liệu merged đã chuẩn hóa từ file `data/merged_supply_weather_clean.parquet`.


In [ ]:
# Load dữ liệu merged đã chuẩn hóa
data_path = os.path.join('..', 'data', 'merged_supply_weather_clean.parquet')

if not os.path.exists(data_path):
    raise FileNotFoundError(f"File không tìm thấy: {data_path}\nVui lòng chạy: python scripts/merge_supplychain_weather.py trước")

df = pd.read_parquet(data_path)
print(f"✓ Đã load {len(df):,} records")
print(f"✓ Số cột: {len(df.columns)}")

# Hiển thị thông tin cơ bản
print("\n=== THÔNG TIN DATASET ===")
df.info()

print("\n=== 5 DÒNG ĐẦU TIÊN ===")
df.head()


## 2. Aggregation & EDA

Aggregate dữ liệu theo tháng và quốc gia, sau đó phân tích time series.


In [ ]:
# Aggregate revenue by month and country
df['order_date'] = pd.to_datetime(df['order date (DateOrders)'])
df['year_month'] = df['order_date'].dt.to_period('M')

# Aggregate by month and country
revenue_ts = df.groupby(['year_month', 'Order Country']).agg({
    'Sales': 'sum',
    'Order Id': 'nunique',  # Number of orders
    'temperature_2m_mean': 'mean',
    'precipitation_sum': 'mean',
    'weather_risk_level': 'mean'
}).reset_index()

revenue_ts.columns = ['year_month', 'country', 'revenue', 'order_count', 
                      'avg_temperature', 'avg_precipitation', 'avg_weather_risk']

# Convert period to datetime
revenue_ts['date'] = revenue_ts['year_month'].dt.to_timestamp()
revenue_ts = revenue_ts.sort_values('date')

print(f"✓ Đã aggregate thành {len(revenue_ts):,} time periods")
print(f"Date range: {revenue_ts['date'].min()} đến {revenue_ts['date'].max()}")

# Hiển thị thông tin
print("\n=== THÔNG TIN TIME SERIES ===")
revenue_ts.info()

print("\n=== 5 DÒNG ĐẦU TIÊN ===")
revenue_ts.head()


In [ ]:
# Visualize time series cho top 5 countries
plt.figure(figsize=(14, 6))
for country in revenue_ts['country'].value_counts().head(5).index:
    country_data = revenue_ts[revenue_ts['country'] == country]
    plt.plot(country_data['date'], country_data['revenue'], label=country, linewidth=2)

plt.title('Doanh thu theo thời gian - Top 5 quốc gia', fontsize=14, fontweight='bold')
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Doanh thu', fontsize=12)
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Phân phối doanh thu
plt.figure(figsize=(10, 6))
plt.hist(revenue_ts['revenue'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Phân phối doanh thu', fontsize=14, fontweight='bold')
plt.xlabel('Doanh thu', fontsize=12)
plt.ylabel('Tần suất', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("=== THỐNG KÊ DOANH THU ===")
print(revenue_ts['revenue'].describe())


## 3. Tiền xử lý & Feature Engineering

**Pipeline xử lý:**
- Tạo lag features (revenue_lag1, lag2, lag3)
- Rolling statistics (moving average 7, 30 tháng, std 7 tháng)
- Time features (month, quarter, year, cyclical encoding)
- Weather features (aggregate)
- Country encoding (one-hot cho top countries)


In [ ]:
# Tạo lag features
revenue_ts['revenue_lag1'] = revenue_ts.groupby('country')['revenue'].shift(1)
revenue_ts['revenue_lag2'] = revenue_ts.groupby('country')['revenue'].shift(2)
revenue_ts['revenue_lag3'] = revenue_ts.groupby('country')['revenue'].shift(3)

# Rolling statistics
revenue_ts['revenue_ma7'] = revenue_ts.groupby('country')['revenue'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)
revenue_ts['revenue_ma30'] = revenue_ts.groupby('country')['revenue'].transform(
    lambda x: x.rolling(window=30, min_periods=1).mean()
)
revenue_ts['revenue_std7'] = revenue_ts.groupby('country')['revenue'].transform(
    lambda x: x.rolling(window=7, min_periods=1).std()
)

# Time features
revenue_ts['month'] = revenue_ts['date'].dt.month
revenue_ts['quarter'] = revenue_ts['date'].dt.quarter
revenue_ts['year'] = revenue_ts['date'].dt.year
revenue_ts['month_sin'] = np.sin(2 * np.pi * revenue_ts['month'] / 12)
revenue_ts['month_cos'] = np.cos(2 * np.pi * revenue_ts['month'] / 12)

# Country encoding (one-hot for top countries)
top_countries = revenue_ts['country'].value_counts().head(10).index
for country in top_countries:
    revenue_ts[f'country_{country}'] = (revenue_ts['country'] == country).astype(int)

print(f"✓ Đã tạo {len(revenue_ts.columns)} features")


In [ ]:
# Chọn features
feature_cols = [
    'revenue_lag1', 'revenue_lag2', 'revenue_lag3',
    'revenue_ma7', 'revenue_ma30', 'revenue_std7',
    'month', 'quarter', 'year', 'month_sin', 'month_cos',
    'avg_temperature', 'avg_precipitation', 'avg_weather_risk',
    'order_count'
] + [f'country_{c}' for c in top_countries]

# Remove rows with NaN (from lag features)
revenue_ts_clean = revenue_ts.dropna(subset=['revenue'] + feature_cols)

X = revenue_ts_clean[feature_cols].fillna(0)
y = revenue_ts_clean['revenue']

print(f"Feature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Số missing values: {X.isnull().sum().sum()}")


## 4. Chia tập train/test

**Tiêu chí chia:** Time-based split (80% train, 20% test) để tránh data leakage.
- Train: 80% dữ liệu đầu tiên theo thời gian
- Test: 20% dữ liệu cuối cùng


In [ ]:
# Time-based split
revenue_ts_clean = revenue_ts_clean.sort_values('date')
split_idx = int(len(revenue_ts_clean) * 0.8)

train_mask = revenue_ts_clean.index[:split_idx]
test_mask = revenue_ts_clean.index[split_idx:]

X_train = X.loc[train_mask]
X_test = X.loc[test_mask]
y_train = y.loc[train_mask]
y_test = y.loc[test_mask]

print(f"Train set: {len(X_train):,} samples")
print(f"Test set: {len(X_test):,} samples")
print(f"\nTrain date range: {revenue_ts_clean.loc[train_mask, 'date'].min()} đến {revenue_ts_clean.loc[train_mask, 'date'].max()}")
print(f"Test date range: {revenue_ts_clean.loc[test_mask, 'date'].min()} đến {revenue_ts_clean.loc[test_mask, 'date'].max()}")


## 5. Huấn luyện mô hình

**Các mô hình sẽ thử:**
- **Baseline:** Linear Regression (đơn giản, dễ interpret)
- **Tree-based:** Random Forest Regressor, XGBoost Regressor (xử lý non-linear, feature importance)


In [ ]:
# 5.1. Linear Regression (Baseline)
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("=== KẾT QUẢ LINEAR REGRESSION ===")
print(f"MAE: {mae_lr:,.2f}")
print(f"RMSE: {rmse_lr:,.2f}")
print(f"MAPE: {mape_lr:.2f}%")
print(f"R²: {r2_lr:.4f}")


In [ ]:
# 5.2. Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("=== KẾT QUẢ RANDOM FOREST ===")
print(f"MAE: {mae_rf:,.2f}")
print(f"RMSE: {rmse_rf:,.2f}")
print(f"MAPE: {mape_rf:.2f}%")
print(f"R²: {r2_rf:.4f}")


In [ ]:
# 5.3. XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=RANDOM_STATE
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mape_xgb = mean_absolute_percentage_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("=== KẾT QUẢ XGBOOST ===")
print(f"MAE: {mae_xgb:,.2f}")
print(f"RMSE: {rmse_xgb:,.2f}")
print(f"MAPE: {mape_xgb:.2f}%")
print(f"R²: {r2_xgb:.4f}")


## 6. Đánh giá mô hình & Trực quan hóa

**Metrics chính:**
- **MAE (Mean Absolute Error):** Lỗi trung bình tuyệt đối
- **RMSE (Root Mean Squared Error):** Lỗi bình phương trung bình (penalize lỗi lớn hơn)
- **MAPE (Mean Absolute Percentage Error):** Lỗi phần trăm trung bình
- **R² (R-squared):** Tỉ lệ phương sai được giải thích


In [ ]:
# So sánh các mô hình
models_dict = {
    'Linear Regression': y_pred_lr,
    'Random Forest': y_pred_rf,
    'XGBoost': y_pred_xgb
}

results = []
for name, pred in models_dict.items():
    results.append({
        'Model': name,
        'MAE': mean_absolute_error(y_test, pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, pred)),
        'MAPE': mean_absolute_percentage_error(y_test, pred),
        'R²': r2_score(y_test, pred)
    })

results_df = pd.DataFrame(results)
print("\n=== SO SÁNH CÁC MÔ HÌNH ===")
print(results_df.to_string(index=False))


In [ ]:
# Scatter plots: Actual vs Predicted
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (name, pred) in enumerate(models_dict.items()):
    axes[idx].scatter(y_test, pred, alpha=0.5, s=20)
    axes[idx].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[idx].set_xlabel('Doanh thu thực tế', fontsize=11)
    axes[idx].set_ylabel('Doanh thu dự báo', fontsize=11)
    axes[idx].set_title(f'{name}\nR² = {r2_score(y_test, pred):.3f}', fontsize=12, fontweight='bold')
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Time series plot: Actual vs Predicted cho top 3 countries
test_dates = revenue_ts_clean.loc[test_mask, 'date']
test_countries = revenue_ts_clean.loc[test_mask, 'country'].unique()[:3]

fig, axes = plt.subplots(len(test_countries), 1, figsize=(14, 5*len(test_countries)))
if len(test_countries) == 1:
    axes = [axes]

for idx, country in enumerate(test_countries):
    country_mask = revenue_ts_clean.loc[test_mask, 'country'] == country
    country_dates = test_dates[country_mask]
    country_actual = y_test[country_mask]
    country_pred = y_pred_xgb[country_mask]
    
    axes[idx].plot(country_dates, country_actual, label='Thực tế', linewidth=2, marker='o')
    axes[idx].plot(country_dates, country_pred, label='Dự báo (XGBoost)', linewidth=2, marker='s', linestyle='--')
    axes[idx].set_title(f'Dự báo doanh thu: {country}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Ngày', fontsize=11)
    axes[idx].set_ylabel('Doanh thu', fontsize=11)
    axes[idx].legend(fontsize=10)
    axes[idx].grid(alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Feature Importance (XGBoost)
if hasattr(xgb_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False).head(15)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_importance, y='feature', x='importance', palette='viridis')
    plt.title('Top 15 Feature Importances (XGBoost)', fontsize=14, fontweight='bold')
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\n=== TOP 10 FEATURES QUAN TRỌNG NHẤT ===")
    print(feature_importance.head(10).to_string(index=False))


## 7. Kết luận & Gợi ý

### Kết quả chính
- **Model tốt nhất:** XGBoost (dựa trên R² và MAPE)
- **Metric chính:** R², MAPE (quan trọng cho business)

### Nhận xét
**Features quan trọng (dựa trên feature importance):**
- `revenue_lag1`: Doanh thu tháng trước là yếu tố quan trọng nhất (autocorrelation)
- `revenue_ma7`, `revenue_ma30`: Moving averages nắm bắt xu hướng dài hạn
- `month_sin`, `month_cos`: Seasonality có ảnh hưởng
- `avg_weather_risk`: Weather impact

### Hạn chế
- ⚠️ Chưa xử lý trend (có thể dùng differencing)
- ⚠️ Thiếu external features (marketing spend, promotions, economic indicators)
- ⚠️ Model chưa được hyperparameter tuning kỹ
- ⚠️ Chưa xử lý outliers trong target

### Hướng phát triển
1. **Time Series Methods:**
   - ARIMA, SARIMA (cho seasonality)
   - Prophet (Facebook)
   - LSTM (Deep Learning)

2. **Feature Engineering:**
   - Trend features (linear, polynomial)
   - Holiday features
   - Economic indicators (GDP, inflation)

3. **Ensemble:**
   - Combine time series models với tree-based models
   - Stacking

4. **Deployment:**
   - Real-time forecasting API
   - Model monitoring (drift detection)
   - A/B testing
